In [3]:
import pandas as pd
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
from io import StringIO
from cleodata.utils.secrets import get_secret
import s3fs
from fastparquet import ParquetFile
from cleodata.sources.sync.sync import SyncDataSource
boto3.setup_default_session(profile_name='DataScientist-878877078763')
redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

2024-05-21 14:14:04 [debug    ] fetching credentials          
2024-05-21 14:14:06 [info     ] Credentials acquired          
2024-05-21 14:14:06 [info     ] Built connection pool         


In [4]:
def read_from_s3(path):
    """Read parquet files and combine them into a single dataframe"""
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{path}*.parquet")

    if len(all_paths_from_s3) > 0:
        s3 = s3fs.S3FileSystem()
        fp_obj = ParquetFile(
            all_paths_from_s3, open_with=s3.open
        )  # use s3fs as the filesystem
        data = fp_obj.to_pandas()
        return data
    elif len(all_paths_from_s3)==1:
        return pd.read_parquet(all_paths_from_s3[0])
    else:
        print(f"Nothing found")
        print(f"paths from a{all_paths_from_s3}")
    
def read_csv_s3(bucket, key):
    try:
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(obj['Body'])
        return df
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print("Key doesn't match. Please check the key value entered.")


def list_s3_flies(base_path):
    fs = s3fs.core.S3FileSystem()
    all_paths_from_s3 = fs.glob(path=f"{base_path}*.parquet")
    return all_paths_from_s3


In [6]:
base_path = 's3://cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_1230'
fs = s3fs.core.S3FileSystem()
all_paths_from_s3 = fs.glob(path=f"{base_path}*.parquet")
all_paths_from_s3

['cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300000_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300001_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300002_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300003_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300004_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300005_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300006_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions__240521_12300007_part_00.parquet',
 'cleo-data-science/transaction_enrichment/experimental_data/caste/transactions_

In [7]:
df_trans = read_from_s3(base_path)

: 

In [ ]:
df_trans.shape

In [ ]:
bucket_name = 'cleo-data-science'
folder_name = 'transaction_enrichment/experimental_data/caste/transactions__240521_12300'

def list_all_files(bucket_name, folder_name):
    s3_client = boto3.client('s3')
    response = s3_client.list_objects_v2(Bucket = bucket_name, Prefix = folder_name)
    print(response)
    if 'Contents' in response:

        # for obj in response['Contents']:
        #     print(obj['Key'])
        all_paths_from_s3   = [obj['Key'] for obj in response['Contents']]
        # all_paths_from_s3 = fs.glob(path=f"{path}*.parquet")
        print(all_paths_from_s3)

        return all_paths_from_s3
    else:
        return None

In [ ]:
import boto3

#session = boto3.Session( aws_access_key_id='<your_access_key_id>', aws_secret_access_key='<your_secret_access_key>')

boto3.setup_default_session(profile_name='DataScientist-878877078763')

session = boto3.Session()
s3 = session.resource('s3')



my_bucket = s3.Bucket('cleo-data-science/transaction_enrichment')

for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object.key)

In [ ]:
print(path)
ppn = 'cleo-data-science/transaction_enrichment/experimental_data/caste/'
fs = s3fs.core.S3FileSystem()
all_paths_from_s3 = fs.glob(path=f"{path}*")
print(all_paths_from_s3)

In [ ]:
fs = s3fs.core.S3FileSystem()
all_paths_from_s3 = fs.glob(path=f"cleo-data-science/transaction_enrichment/experimental_data/caste/transactions*")
all_paths_from_s3

# Copy all the model data to an s3 bucket

In [3]:
local_directory = "/home/sagemaker-user/exploration/MERCHANTS/models/model3"

In [4]:
#copy 
import os
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

def upload_directory_to_s3(local_directory, bucket_name, s3_directory):
    s3_client = boto3.client('s3')

    for root, dirs, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, local_directory)
            s3_path = os.path.join(s3_directory, relative_path)

            try:
                s3_client.upload_file(local_path, bucket_name, s3_path)
                print(f'Successfully uploaded {local_path} to s3://{bucket_name}/{s3_path}')
            except FileNotFoundError:
                print(f'The file {local_path} was not found')
            except NoCredentialsError:
                print('Credentials not available')
            except PartialCredentialsError:
                print('Incomplete credentials provided')

# Example usage
#local_directory = "models/model3"
bucket_name = 'cleo-data-science'
s3_directory = "transaction_enrichment/experimental_data/caste/pairs_model/model3"

upload_directory_to_s3(local_directory, bucket_name, s3_directory)

Successfully uploaded /home/sagemaker-user/exploration/MERCHANTS/models/model3/runs/Jun04_15-51-54_default/events.out.tfevents.1717516353.default.3432.0 to s3://cleo-data-science/transaction_enrichment/experimental_data/caste/pairs_model/model3/runs/Jun04_15-51-54_default/events.out.tfevents.1717516353.default.3432.0
Successfully uploaded /home/sagemaker-user/exploration/MERCHANTS/models/model3/runs/Jun04_15-58-41_default/events.out.tfevents.1717516784.default.5391.0 to s3://cleo-data-science/transaction_enrichment/experimental_data/caste/pairs_model/model3/runs/Jun04_15-58-41_default/events.out.tfevents.1717516784.default.5391.0
Successfully uploaded /home/sagemaker-user/exploration/MERCHANTS/models/model3/runs/Jun04_16-21-27_default/events.out.tfevents.1717518089.default.10120.0 to s3://cleo-data-science/transaction_enrichment/experimental_data/caste/pairs_model/model3/runs/Jun04_16-21-27_default/events.out.tfevents.1717518089.default.10120.0
Successfully uploaded /home/sagemaker-use